In [2]:
!pip install langchain_community langchain_chroma

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os



In [2]:
load_dotenv()
api_key = os.getenv('OpenAI_API_Key')

In [3]:
# 마크다운 파일 경로
file_path = 'rag_example.jsonl'

loader = TextLoader(file_path)
documents = loader.load()

In [4]:
chunk_size = 300
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(documents)

In [5]:
splits

[Document(metadata={'source': 'rag_example.jsonl'}, page_content='{"instruction":"양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?","output":"양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \\n\\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\\n\\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \\n\\n고구마'),
 Document(metadata={'source': 'rag_example.jsonl'}, page_content='것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\\n\\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \\n\\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.","url":"https:\\/\\/kin.naver.com\\/qna\\/detail.naver?d1id=11&dirId=1116&docId=55320268"}'),
 Document(metadata={'source': 'rag_example.jsonl'}, page_content='{"instruction":"스웨터의 유래는 어디에서 시작되었나요?","output":"스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서

In [6]:
# 생성 모델 초기화
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=api_key))
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [9]:
prompt = '''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
You must print only in retrived context about the questions. Don't print any idea not retrieved data. Be flexible with your questions. (Example: What is React? -> This is a question about reAct prompting in Prompt Engineering, Not React framework.)
When you print answer, you should print answer in json returning 2 parameters: 'activate_RAG' and 'Explain'. If you can explain users question in retrieved context, the value of 'activate_RAG' is 'Yes'. If you can't, return 'No'.
And if 'activate_RAG' value is 'Yes' return the answer in 'Explain' parts, 'activate_RAG' is 'No', Print 'None'.
마지막으로, 답변을 출력할때는 한글로 출력하세요. 그리고 정해진 형식에 맞게 json 파일로 출력해야함을 잊지 마세요.


Question: {question}

Context: {context}

[Example]
"activate_RAG": "No",
"Explain": "I don't know"

Answer:'''

prompt_temp = PromptTemplate.from_template(prompt)

In [10]:
# 생성 모델 초기화
llm = OpenAI(api_key=api_key)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_temp
    | llm
    | StrOutputParser()
)

/home/jyh9701/prompt/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
result = rag_chain.invoke("광해군에 대해서 설명해줘?")

In [16]:
result

' {"activate_RAG": "Yes", "explain": "광해군이 폭군인가에 대해서는 서인, 인조, 인목대비 등의 시각에서는 폭군이라고 말하지만, 역사적으로는 패자이기 때문에 그들의 시각만 볼 수는 없습니다. 광해군의 업적으로는 산림 숭용, 실리 외교, 경기도 대동법 시행, 양전 사업 등이 있습니다. 이 외에도 궁궐 수리와 건립, 서적 간행, 후금과의 협상 등 광해군의 업적은 매우 다양합니다."}'